# US COVID-19 Data

In [2]:
import os
import sys

from IPython.display import IFrame
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
# import missingno as msno
# import seaborn as sns
# import matplotlib.patches as patches
# import warnings
# import networkx as nx
# import osmnx as ox
# import requests
# import matplotlib.cm as cm
# import matplotlib.colors as colors

import urllib
import zipfile
import glob

## Import data from NYTimes

* Github: https://github.com/nytimes/covid-19-data

In [4]:
# from IPython.display import IFrame

IFrame(src='https://www.nytimes.com/interactive/2020/us/coronavirus-us-cases.html', width=800, height=500)

### Read in US, States, and Counties data

In [19]:
nytimes_us_url='https://raw.githubusercontent.com/nytimes/covid-19-data/master/us.csv'
nytimes_states_url='https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-states.csv'
nytimes_counties_url='https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv'

### US

In [21]:
nytimes_us_df = pd.read_csv(nytimes_us_url)

nytimes_us_df.head()

,date,cases,deaths
0,2020-01-21,1,0
1,2020-01-22,1,0
2,2020-01-23,1,0
3,2020-01-24,2,0
4,2020-01-25,3,0


In [22]:
nytimes_us_df.tail()

,date,cases,deaths
93,2020-04-23,868019,44516
94,2020-04-24,904604,46503
95,2020-04-25,938752,48325
96,2020-04-26,965214,49464
97,2020-04-27,988250,50819


### States

In [13]:
nytimes_states_df = pd.read_csv(nytimes_states_url)

nytimes_states_df.head()

,date,state,fips,cases,deaths
0,2020-01-21,Washington,53,1,0
1,2020-01-22,Washington,53,1,0
2,2020-01-23,Washington,53,1,0
3,2020-01-24,Illinois,17,1,0
4,2020-01-24,Washington,53,1,0


In [14]:
nytimes_states_df.tail()

,date,state,fips,cases,deaths
3089,2020-04-27,Virginia,51,13535,458
3090,2020-04-27,Washington,53,13864,771
3091,2020-04-27,West Virginia,54,1077,37
3092,2020-04-27,Wisconsin,55,6081,281
3093,2020-04-27,Wyoming,56,389,7


### Counties

In [11]:
nytimes_counties_df = pd.read_csv(nytimes_counties_url)

nytimes_counties_df.head()

,date,county,state,fips,cases,deaths
0,2020-01-21,Snohomish,Washington,53061.0,1,0
1,2020-01-22,Snohomish,Washington,53061.0,1,0
2,2020-01-23,Snohomish,Washington,53061.0,1,0
3,2020-01-24,Cook,Illinois,17031.0,1,0
4,2020-01-24,Snohomish,Washington,53061.0,1,0


In [12]:
nytimes_counties_df.tail()

,date,county,state,fips,cases,deaths
95415,2020-04-27,Sublette,Wyoming,56035.0,1,0
95416,2020-04-27,Sweetwater,Wyoming,56037.0,10,0
95417,2020-04-27,Teton,Wyoming,56039.0,64,1
95418,2020-04-27,Uinta,Wyoming,56041.0,6,0
95419,2020-04-27,Washakie,Wyoming,56043.0,4,0


In [8]:
nytimes_df.shape

(95420, 6)

## Import data from COVID-19 Tracking Project

* API: https://covidtracking.com/api

In [18]:
IFrame(src='https://covidtracking.com/data', width=800, height=500)

### Read in US Historical Data and States Historical Data

In [30]:
covtrack_us_hist_url = 'https://covidtracking.com/api/v1/us/daily.csv'
covtrack_us_now_url = 'https://covidtracking.com/api/v1/us/current.csv'

covtrack_states_hist_url = 'https://covidtracking.com/api/v1/states/daily.csv'
covtrack_states_now_url = 'https://covidtracking.com/api/v1/states/current.csv'

### US

In [31]:
covtrack_us_now_df = pd.read_csv(covtrack_us_now_url)

covtrack_us_now_df

,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,onVentilatorCumulative,recovered,hash,lastModified,death,hospitalized,total,totalTestResults,posNeg,notes
0,1005592,4790136,4206,54940,106265,9491,3798,4760,195,139299,874d58c394c4c87aff46853b239edde8843c39da,4/29/2020,52525,106265,5799934,5795728,5795728,"NOTE: ""total"", ""posNeg"", ""hospitalized"" will b..."


In [32]:
covtrack_us_hist_df = pd.read_csv(covtrack_us_hist_url)

covtrack_us_hist_df.head()

,date,states,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,...,hospitalized,total,totalTestResults,posNeg,fips,deathIncrease,hospitalizedIncrease,negativeIncrease,positiveIncrease,totalTestResultsIncrease
0,20200428,56,1005592,4790136.0,4206.0,54940.0,106308.0,9491.0,3798.0,4760.0,...,106308.0,5799934,5795728,5795728,NaN,2198.0,2059.0,177775.0,24458.0,202233.0
1,20200427,56,981134,4612361.0,4077.0,54971.0,104249.0,9655.0,3720.0,4867.0,...,104249.0,5597572,5593495,5593495,NaN,1163.0,3016.0,168567.0,21876.0,190443.0
2,20200426,56,959258,4443794.0,4445.0,55050.0,101233.0,14831.0,2571.0,5121.0,...,101233.0,5407497,5403052,5403052,NaN,1095.0,2134.0,179387.0,27016.0,206403.0
3,20200425,56,932242,4264407.0,5315.0,56344.0,99099.0,15020.0,2516.0,5266.0,...,99099.0,5201964,5196649,5196649,NaN,1818.0,2422.0,235320.0,35936.0,271256.0
4,20200424,56,896306,4029087.0,4396.0,56075.0,96677.0,15078.0,2468.0,5187.0,...,96677.0,4929789,4925393,4925393,NaN,1866.0,3407.0,197707.0,34518.0,232225.0


In [33]:
covtrack_us_hist_df.shape

(98, 25)

### States

In [34]:
covtrack_states_now_df = pd.read_csv(covtrack_states_now_url)

covtrack_states_now_df.head()

,state,positive,positiveScore,negativeScore,negativeRegularScore,commercialScore,grade,score,notes,dataQualityGrade,...,checkTimeEt,death,hospitalized,total,totalTestResults,posNeg,fips,dateModified,dateChecked,hash
0,AK,351,1.0,1.0,1.0,1.0,A,4.0,"Please stop using the ""total"" field. Use ""tota...",B,...,4/28 15:14,9,NaN,17089,17089,17089,2,2020-04-28T18:50:00Z,2020-04-28T19:14:00Z,f5f8fd60d9ab9000b01078ebb64a6ead70991838
1,AL,6687,1.0,1.0,0.0,1.0,B,3.0,"Please stop using the ""total"" field. Use ""tota...",B,...,4/28 16:23,242,911.0,75827,75827,75827,1,2020-04-28T04:00:00Z,2020-04-28T20:23:00Z,ce913fed81a2519acc5fad3fa47a46d5de9791a7
2,AR,3111,1.0,1.0,1.0,1.0,A,4.0,"Please stop using the ""total"" field. Use ""tota...",B,...,4/28 15:15,52,NaN,40671,40671,40671,5,2020-04-28T13:00:00Z,2020-04-28T19:15:00Z,db047c4c735a3fed6ceec83410b67647820351dc
3,AZ,6948,1.0,1.0,0.0,1.0,B,3.0,"Please stop using the ""total"" field. Use ""tota...",A+,...,4/28 14:04,293,1095.0,67438,67438,67438,4,2020-04-27T04:00:00Z,2020-04-28T18:04:00Z,ed30f69fbcb735ea556fd73c6527c52e2ea64d95
4,CA,45031,1.0,1.0,0.0,1.0,B,3.0,"Please stop using the ""total"" field. Use ""tota...",B,...,4/28 16:03,1809,NaN,577608,577608,577608,6,2020-04-27T04:00:00Z,2020-04-28T20:03:00Z,6465238e0ca54eddc1da1a09b2a529005bdefba6


In [35]:
covtrack_states_now_df.shape

(56, 30)

In [26]:
covtrack_states_hist_df = pd.read_csv(covtrack_states_hist_url)

covtrack_states_hist_df.head()

,date,state,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,...,hospitalized,total,totalTestResults,posNeg,fips,deathIncrease,hospitalizedIncrease,negativeIncrease,positiveIncrease,totalTestResultsIncrease
0,20200428,AK,351.0,16738.0,NaN,16.0,NaN,NaN,NaN,NaN,...,NaN,17089.0,17089.0,17089.0,2,0.0,0.0,827.0,6.0,833.0
1,20200428,AL,6687.0,69140.0,NaN,NaN,911.0,NaN,335.0,NaN,...,911.0,75827.0,75827.0,75827.0,1,20.0,39.0,1642.0,188.0,1830.0
2,20200428,AR,3111.0,37560.0,NaN,104.0,NaN,NaN,NaN,20.0,...,NaN,40671.0,40671.0,40671.0,5,2.0,0.0,120.0,94.0,214.0
3,20200428,AS,0.0,3.0,17.0,NaN,NaN,NaN,NaN,NaN,...,NaN,20.0,3.0,3.0,60,0.0,0.0,0.0,0.0,0.0
4,20200428,AZ,6948.0,60490.0,NaN,737.0,1095.0,303.0,NaN,193.0,...,1095.0,67438.0,67438.0,67438.0,4,18.0,44.0,663.0,232.0,895.0


In [28]:
covtrack_states_hist_df.shape

(3041, 25)